In [ ]:
! python -m spacy download en_core_web_lg

In [1]:
import base64

In [2]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [3]:
import re
import spacy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json

In [41]:
nlp = spacy.load('en_core_web_sm')

In [57]:
#Function to extract names from the string using spacy
def extract_name(string):
    names = []
#     nlp = spacy.load('en')
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(string)
    ent_list = ['PERSON']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            names.append(ent.text)
    return np.unique(names).tolist()

In [39]:
def extract_org(string):
    org = []
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(string)
    ent_list = ['GPE','NORP','ORG','LOC','PRODUCT']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            org.append(ent.text)
    return np.unique(org).tolist()

In [7]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = np.unique(r.findall(string))
    return [re.sub(r'\D', '', number) for number in phone_numbers]

In [8]:
#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return np.unique(r.findall(string)).tolist()
    #return r.findall(string)

In [40]:
def get_nouns(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    nouns_list=[]
    for token in doc:
        if token.pos_ == 'PROPN':
            nouns_list.append(token.text)
#     print(re.sub(' +', ' ', ' '.join(nouns_list)))
    return re.sub(' +', ' ', ' '.join(nouns_list))

In [116]:
def extract_twitter(string):
    r = re.compile(r'twitter.com/([^\s:]+)')
    return r.findall(string)

def extract_linkedin(string):
    r = re.compile(r'linkedin.com/([^\s:]+)')
    return r.findall(string)

def extract_facebook(string):
    r = re.compile(r'facebook.com/([^\s:]+)')
    return r.findall(string)

In [10]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


In [ ]:
def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])

In [ ]:
main()

In [19]:
with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

In [20]:
if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

In [21]:
service = build('gmail', 'v1', credentials=creds)

In [98]:
service.users().messages().list(userId='me').execute()

{'messages': [{'id': '16ae4c77077bc237', 'threadId': '16ae4b0f524b19bb'},
  {'id': '16ae4b0f524b19bb', 'threadId': '16ae4b0f524b19bb'},
  {'id': '16ae4af2960cb89a', 'threadId': '16ae4af2960cb89a'},
  {'id': '16ae4aea1568e2ef', 'threadId': '16ae4aea1568e2ef'},
  {'id': '16ae45f7e57ac19c', 'threadId': '16ae45f7e57ac19c'},
  {'id': '16ae448efec5b907', 'threadId': '16ae448efec5b907'},
  {'id': '16ae3f93c5e4d089', 'threadId': '16ae3f93c5e4d089'},
  {'id': '16ae3f615ec45be6', 'threadId': '16ae3f615ec45be6'},
  {'id': '16ae3d82347e2e6f', 'threadId': '16ae3d82347e2e6f'},
  {'id': '16ae3d6b5cf72005', 'threadId': '16ae3d6b5cf72005'},
  {'id': '16ae3ceb6a93290a', 'threadId': '16ae3ceb6a93290a'},
  {'id': '16ae380ec2856525', 'threadId': '16ae380ec2856525'},
  {'id': '16ae36773e96cef0', 'threadId': '16ae36773e96cef0'},
  {'id': '16ae367406a2bad0', 'threadId': '16ae367406a2bad0'},
  {'id': '16ae3570d0384855', 'threadId': '16ae3570d0384855'},
  {'id': '16ae34f3734e16b4', 'threadId': '16ae34f3734e16b4

In [99]:
msgs = service.users().messages().get(userId='me').execute()

In [ ]:
next_page = service.users().messages().list_next(service.users().messages().list(userId='me'), msgs)
next_page.execute()

{'From': 'Shoumik Goswami <shoumikgoswami@gmail.com>', 'To': 'Diljeet Singh <diljeets1994@gmail.com>', 'Subject': 'Re: Test Mail', 'Date': 'Sun, 19 May 2019 11:28:32 +0530'}

In [25]:
def response_extraction(email_data):
    response = {}
    headers_data = {'From':'', 'To':'', 'Subject':'', 'Date':'', 'Cc':''}
#     headers = ['From', 'To', 'Subject', 'Date', 'Cc']
    for data in email_data['payload']['headers']:
        if data['name'] in headers_data:
            headers_data[data['name']] = data['value']
    response['Subject'] = headers_data['Subject']
    response['Date'] = headers_data['Date']
    

In [26]:
def extract_body(email_data):
    try:
        if ((email_data['payload']['body']['size']==0) and ('parts' in email_data['payload']['parts'][0])):
            for body_data in email_data['payload']['parts'][0]['parts']:
                if body_data['mimeType'] == 'text/html':
                    print("body_data in email_data['payload']['parts'][0]['parts']")
        elif ((email_data['payload']['body']['size']==0) and ('data' in email_data['payload']['parts'][0]['body'])):
            for body_data in email_data['payload']['parts']:
                if body_data['mimeType'] == 'text/html':
                    print("email_data['payload']['parts']")
        elif ((email_data['payload']['body']['size']==0) and ('attachmentId' in email_data['payload']['parts'][0]['body'])):
            pass
        else:
            if email_data['payload']['mimeType'] == 'text/html':
                print("email_data['payload']['body']['data']")
    except Exception as e:
        print(e)

In [27]:
# cleaning text and returning meaningful tokens
import string
import unicodedata
from nltk.corpus import stopwords

def text_cleaning(text):
#     text = [word for word in text.split(' ') if word.lower() not in stopwords.words('english')]
#     text =''.join(nopunc)
    
    # removing non unicode
    text = unicodedata.normalize("NFKD", text)
    #Removing commas and spaces in the resume for an effecient check
#     text = text.replace(',',' ')
#     text = text.replace('\\n',' ')
#     text = text.replace('\\r',' ')
#     text = text.replace('\\r\\n',' ')
#     text = text.replace('*','')
    
    bad_characters = ['\r', '\n', '\\r\\n', ',', '*', ';']
    nopunc = [" " if char in bad_characters else char for char in text]
    text =''.join(nopunc)
    text = re.sub(' +', ' ', text)
    bad_words= ['&gt;', '&gt', '&lt']
    text = text.split(' ')
    text = ['' if word in bad_words else word for word in text]
    text =' '.join(text)
#     # removing punctuation
#     nopunc = [char for char in text if char not in string.punctuation]
#     text =''.join(nopunc)
    # removing stop words
    
    return text

16ad4dd20e0cc588 - courtney


In [ ]:
service.users().messages().list(userId='me').execute()

In [ ]:
for msg in msgs['messages']:
    print(msg['id'])
    email_data = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
    email_data['payload']
    extract_body(email_data)
    

In [142]:
# email_data = service.users().messages().get(userId='me', id='16ae4c77077bc237', format='full').execute()
email_data = service.users().messages().get(userId='me', id='16ad4dd20e0cc588').execute()
email_data['payload']
# [0]['parts'][0]['body']['data']
# ['parts'][0]['body']
soup = BeautifulSoup(base64.urlsafe_b64decode(email_data['payload']['parts'][0]['body']['data']), 'html.parser')
# email_text = soup.get_text()
# email_text
email_text = soup.decode_contents()
email_text

'Trail +1\r\n\r\nOn Sun, May 19, 2019 at 11:27 AM Shoumik Goswami <shoumikgoswami@gmail.com>\r\nwrote:\r\n\r\n&gt;\r\n&gt; Shoumik Goswami\r\n&gt;\r\n&gt;\r\n&gt; ---------- Forwarded message ---------\r\n&gt; From: Courtenay Farquharson <courtenay.farquharson@gmail.com>\r\n&gt; Date: Sat, May 18, 2019 at 4:17 PM\r\n&gt; Subject: Fwd: 119050425000150\r\n&gt; To: <shoumikgoswami@gmail.com>\r\n&gt;\r\n&gt;\r\n&gt;\r\n&gt;\r\n&gt; ---------- Forwarded message ---------\r\n&gt; From: Nasmin Kaisar (MINDTREE LIMITED) <v-naskai@microsoft.com>\r\n&gt; Date: Sat, May 18, 2019 at 9:08 AM\r\n&gt; Subject: RE: 119050425000150\r\n&gt; To: Courtenay Farquharson <courtenay.farquharson@gmail.com>\r\n&gt; Cc: Elyaas Mohammed (MINDTREE LIMITED) <v-elmoha@microsoft.com>\r\n&gt;\r\n&gt;\r\n&gt; Hello Courtenay,\r\n&gt;\r\n&gt;\r\n&gt;\r\n&gt; I have processed the refund from my end .\r\n&gt;\r\n&gt; It usually takes 7-10 business days for the refund to be credited back to\r\n&gt; your account.\r\n&gt;\r\

In [111]:
email_data

{'historyId': '5614381',
 'id': '16ae4c77077bc237',
 'internalDate': '1558616435000',
 'labelIds': ['IMPORTANT', 'SENT'],
 'payload': {'body': {'size': 0},
  'filename': '',
  'headers': [{'name': 'MIME-Version', 'value': '1.0'},
   {'name': 'Date', 'value': 'Thu, 23 May 2019 18:30:35 +0530'},
   {'name': 'References',
    'value': '<CAL5Lp9W6cfGes=OignYGgVQTwo9_pQnfY_wbBbp_Te+v7ZutGA@mail.gmail.com> <CAGLgaGGohMkO+B8gLLQBHS_CTxa6i-9Be8CzC3WhkchqoK8Mjw@mail.gmail.com>'},
   {'name': 'In-Reply-To',
    'value': '<CAGLgaGGohMkO+B8gLLQBHS_CTxa6i-9Be8CzC3WhkchqoK8Mjw@mail.gmail.com>'},
   {'name': 'Message-ID',
    'value': '<CAJ5Vba-EJXrkGuUd02XrENbUHwxDgmOmbz45f-RT2Sf5z5BHFA@mail.gmail.com>'},
   {'name': 'Subject', 'value': 'Re: SigParser email'},
   {'name': 'From', 'value': 'Diljeet Singh <diljeets1994@gmail.com>'},
   {'name': 'To',
    'value': 'Shoumik Goswami <shoumikgoswami@gmail.com>, Diljeet Singh <diljeets1994@gmail.com>'},
   {'name': 'Content-Type',
    'value': 'multipart/a

In [113]:
data = text_cleaning(email_text)
data

'Hi Paul Mendoza <pmendoza@sigparser.com> On Thu May 23 2019 at 6:06 PM Shoumik Goswami <shoumikgoswami@gmail.com> wrote:   Shoumik Goswami    ---------- Forwarded message ---------  From: Paul Mendoza <pmendoza@sigparser.com>  Date: Sun May 19 2019 at 12:34 AM  Subject: SigParser email  To: <shoumikgoswami@gmail.com>    Hi   The email you submitted is plain text but you clicked the checkbox for “Is  Html”. Uncheck that checkbox and try again.   Also I am seeing a bunch of escape sequences. You need to unescape those  before you use the API. Look for a Javascript unescape tool.   Let me know if you have any other questions or would like a quick call to  review how to integrate.   Paul  Sent from my iPhone  --  Paul Mendoza  SigParser Founder CEO  760-917-3753   Sent from Gmail Mobile  </shoumikgoswami@gmail.com></pmendoza@sigparser.com></shoumikgoswami@gmail.com></pmendoza@sigparser.com>'

In [ ]:
extract_email_addresses(email_text)

In [ ]:
extract_email_addresses(data)

In [ ]:
extract_name(data)

In [ ]:
extract_name(email_text)

In [ ]:
extract_org(email_text)

In [ ]:
extract_org(data)

In [ ]:
extract_phone_numbers(data)

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(email_text)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
response = {'From':'', 'To':'', 'Subject':'', 'Date':''}
for data in email_data['payload']['headers']:
    if data['name'] in response:
        response[data['name']] = data['value']
response

In [ ]:
for msg_id in msgs['messages']:
    response_data = {'From':'', 'To':'', 'Subject':'', 'Date':''}
    email_data = service.users().messages().get(userId='me', id=msg_id['id'], format='full').execute()
    for data in email_data['payload']['headers']:
        if data['name'] in response_data:
            response_data[data['name']] = data['value']
    print()
    print(response_data)
    

In [ ]:
import base64
counter = 0
try:
    for msg_id in msgs['messages']:
        try:
            email_data = service.users().messages().get(userId='me', id=msg_id['id'], format='full').execute()
            if ((email_data['payload']['body']['size']==0) and ('parts' in email_data['payload']['parts'][0])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts'][0]['parts']:
                    base64.urlsafe_b64decode(body_data['body']['data'])
            elif ((email_data['payload']['body']['size']==0) and ('data' in email_data['payload']['parts'][0]['body'])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts']:
                    base64.urlsafe_b64decode(body_data['body']['data'])
            elif ((email_data['payload']['body']['size']==0) and ('attachmentId' in email_data['payload']['parts'][0]['body'])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts']:
                    base64.urlsafe_b64decode(body_data['body']['attachmentId'])
            else:
                print(msg_id['id'])
                base64.urlsafe_b64decode(body_data['body']['data'])
        except Exception as e:
            print(e)
except Exception as e:
    print(e)
        
        

In [ ]:
email_data = service.users().messages().get(userId='me', id='16ac694f2e874497', format='full').execute()
print(email_data.keys())
email_data['payload']['parts'][0]['parts'][0]['body']['data']

In [ ]:
base64.urlsafe_b64decode(email_data['payload']['parts'][0]['parts'][0]['body']['data'])

# Response

{
	"error": "string",
	"contacts": [{
		"firstName": "string",
		"lastName": "string",
		"emailAddress": "string",
		"emailAddressDomain": "string",
		"emailAddressDomainWithoutTLD": "string",
		"phoneNumber": "string",
		"mobilePhone": "string",
		"voipPhone": "string",
		"officePhone": "string",
		"fax": "string",
		"address": "string",
		"title": "string",
		"twitterUrl": "string",
		"twitterHandle": "string",
		"linkedInUrl": "string",
		"linkedInHandle": "string",
		"companyName": "string",
		"website": "string"
	}],
	"isSpammyLookingEmailMessage": true,
	"isSpammyLookingSender": true,
	"isSpam": true,
	"from_LastName": "string",
	"from_FirstName": "string",
	"from_Fax": "string",
	"from_Phone": "string",
	"from_Address": "string",
	"from_Title": "string",
	"from_MobilePhone": "string",
	"from_OfficePhone": "string",
	"from_LinkedInUrl": "string",
	"from_TwitterUrl": "string",
	"from_TwitterHandle": "string",
	"from_EmailAddress": "string",
	"emails": [{
		"from_EmailAddress": "string",
		"from_Name": "string",
		"textBody": "string",
		"htmlLines": ["string"],
		"date": "2019-05-05T22:27:56.124Z",
		"didParseCorrectly": true,
		"to": [{
			"name": "string",
			"emailAddress": "string"
		}],
		"cc": [{
			"name": "string",
			"emailAddress": "string"
		}],
		"htmlBody": "string",
		"spammyLookingEmail": true,
		"subject": "string",
		"cleanedBodyHtml": "string",
		"cleanedBodyPlain": "string"
	}],
	"from_LinkedInHandle": "string",
	"duration": 0,
	"cleanedemailbody": "string",
	"cleanedemailbody_ishtml": true,
	"cleanedemailbody_plain": "string",
	"from_CompanyName": "string",
	"from_Website": "string",
	"from_EmailAddressDomain": "string",
	"from_EmailAddressDomainWithoutTLD": "string"
}

In [91]:
email_data['payload']['parts'

[{'body': {'data': 'U2hvdW1payBHb3N3YW1pDQoNCg0KLS0tLS0tLS0tLSBGb3J3YXJkZWQgbWVzc2FnZSAtLS0tLS0tLS0NCkZyb206IFBhdWwgTWVuZG96YSA8cG1lbmRvemFAc2lncGFyc2VyLmNvbT4NCkRhdGU6IFN1biwgTWF5IDE5LCAyMDE5IGF0IDEyOjM0IEFNDQpTdWJqZWN0OiBTaWdQYXJzZXIgZW1haWwNClRvOiA8c2hvdW1pa2dvc3dhbWlAZ21haWwuY29tPg0KDQoNCkhpLA0KDQpUaGUgZW1haWwgeW91IHN1Ym1pdHRlZCBpcyBwbGFpbiB0ZXh0IGJ1dCB5b3UgY2xpY2tlZCB0aGUgY2hlY2tib3ggZm9yIOKAnElzDQpIdG1s4oCdLiBVbmNoZWNrIHRoYXQgY2hlY2tib3ggYW5kIHRyeSBhZ2Fpbi4NCg0KQWxzbywgSSBhbSBzZWVpbmcgYSBidW5jaCBvZiBlc2NhcGUgc2VxdWVuY2VzLiBZb3UgbmVlZCB0byB1bmVzY2FwZSB0aG9zZQ0KYmVmb3JlIHlvdSB1c2UgdGhlIEFQSS4gTG9vayBmb3IgYSBKYXZhc2NyaXB0IHVuZXNjYXBlIHRvb2wuDQoNCkxldCBtZSBrbm93IGlmIHlvdSBoYXZlIGFueSBvdGhlciBxdWVzdGlvbnMgb3Igd291bGQgbGlrZSBhIHF1aWNrIGNhbGwgdG8NCnJldmlldyBob3cgdG8gaW50ZWdyYXRlLg0KDQpQYXVsDQpTZW50IGZyb20gbXkgaVBob25lDQotLSANClBhdWwgTWVuZG96YQ0KU2lnUGFyc2VyIEZvdW5kZXIgQ0VPDQo3NjAtOTE3LTM3NTMNCg0KU2VudCBmcm9tIEdtYWlsIE1vYmlsZQ0K',
   'size': 690},
  'filename': '',
  'headers': [{'name':

In [95]:
email_data['payload']['parts'][0]['body']['data']
soup = BeautifulSoup(base64.urlsafe_b64decode(email_data['payload']['parts'][0]['body']['data']), 'html.parser')
content = soup.get_text()
text_cleaning(content)

'Shoumik Goswami ---------- Forwarded message --------- From: Paul Mendoza Date: Sun May 19 2019 at 12:34 AM Subject: SigParser email To: Hi The email you submitted is plain text but you clicked the checkbox for “Is Html”. Uncheck that checkbox and try again. Also I am seeing a bunch of escape sequences. You need to unescape those before you use the API. Look for a Javascript unescape tool. Let me know if you have any other questions or would like a quick call to review how to integrate. Paul Sent from my iPhone -- Paul Mendoza SigParser Founder CEO 760-917-3753 Sent from Gmail Mobile '

In [154]:
email_body= email_data['payload']['parts'][0]['body']['data']
sender_email = 'shoumikgoswami@gmail.com'
sender_name= 'Shoumik Goswami'

response(sender_email,sender_name,email_body)

[{'name': ['Shoumik', 'Goswami'], 'email': 'shoumikgoswami@gmail.com'}, {'name': ['Courtenay', 'Farquharson'], 'email': 'courtenay.farquharson@gmail.com'}, {'name': ['Courtenay', 'Farquharson'], 'email': 'courtenay@parserr.com'}]


In [162]:
import re
s = 'My name is Conrad, and blahblah@gmail.com is my email.'
re.search("@\w+", s).group().replace('@', '')

'gmail'

In [153]:
def response(sender_email,sender_name,email_body):
    contacts = []
    contacts.append({'name':sender_name.split(' ', 1), 'email':sender_email})
    
    html_data = email_body
    soup = BeautifulSoup(base64.urlsafe_b64decode(html_data), 'html.parser')
    content = soup.get_text()
    content = text_cleaning(content)
    
    person_data = data_splitting(content)
    for contact in person_data:
        contacts.append({'name':contact[0], 'email':contact[1], 'domain': re.search("@[\w.]+", contact[1]).group(), 
                         'DomainWithoutTLD':re.search("@\w+", s).group().replace('@', ''), 'mobilePhone'})
    print(contacts)
#     for contact in contacts:
        
        
    
        
    

In [120]:
data1 = '---------- Forwarded message ---------\r\nFrom: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com> 8752365478 \r\nDate: Fri, May 10, 2019 at 5:09 PM\r\nSubject: RE: Dangerous conditions for cycling Akuna (anticlockwise) this\r\nSunday 12th May\r\nTo: george lau <georgeandkim@gmail.com>, Adrian See <adriansee@hotmail.com>,\r\nAdrian Wong <adwong_au@yahoo.com>, Alex Perdomo <alejoperdomo68@gmail.com>,\r\nAndrew Linfoot <andrew.linfoot@gmail.com> 9852634785, Andrew McPherson <\r\nandrewmcp@optusnet.com.au>, A&S <andsal@optusnet.com.au>, Andy Kuo <\r\nandy.kuo1@bayer.com>, Chi Ooi <chigooi@gmail.com>, Chris and Wendy Annable <\r\ncwannable@hotmail.com>, Courtenay Farquharson <\r\ncourtenay.farquharson@gmail.com>, Daniel Lu <mail2danlu@gmail.com>, David\r\nChen <dst.chen@gmail.com>, Dave Jackson <DAVID.A.JACKSON.1138@googlemail.com>,\r\nDavid Stanfield <david.stanfield@hotmail.com>, denis gradinscak <\r\ndgrad76@gmail.com>, Gavin Gillett <gavin@gillettproperties.com.au>, Ian\r\nPartis <ian_partis@optusnet.com.au>, James Ward <james.ward@nbrsap.com.au>,\r\nJohn Cormican <johncormican@icloud.com>, Kerem Kaner <kaner.kerem@gmail.com> 5555555555,\r\nMark Latham <mark.latham@bayer.com>, Maros Elsik <melsik@me.com>,\r\nmatt_bywaters@yahoo.co.uk <matt_bywaters@yahoo.co.uk>,\r\nmichael.aitkenhead@cantab.net <michael.aitkenhead@cantab.net>, Michael Ee <\r\nDaddy_ee@hotmail.com>, Nick Benton <nzrheum@gmail.com>, Peter Janssen <\r\npj@janssen.net.au>, Richard Duggan <rdugganrduggan@yahoo.com.au>, robert\r\nhughes <robertthughes@yahoo.com>, Steve WIlson <wilson_steve@me.com>, Tung\r\nVu <tungvu@bigpond.com>, Tim Lu <tim.rheumatology@gmail.com>, Bennett,\r\nAndrew(Property) <Andrew.Bennett2@anz.com>, Victor Tiong <\r\nvictor.tiong@gmail.com>, YL Wang <wyl03@yahoo.com>, scott girard <\r\nscottrgirard@gmail.com>, Wilson Tehkh <wilson_tehkh@hotmail.com>\r\n\r\n\r\nPeter \xe2\x80\x93 I will see you there. Looks like a cheap coffee run afterwards ;-)\r\n\r\n\r\n\r\nBest regards,\r\n\r\n\r\n*Paul Donnelley *\r\nTel. +61298424752 | Mobile +61417696638\r\n\r\n*From:* george lau <georgeandkim@gmail.com>\r\n*Sent:* Friday, 10 May 2019 4:03 PM\r\n*To:* Adrian See <adriansee@hotmail.com>; Adrian Wong <adwong_au@yahoo.com>;\r\nAlex Perdomo <alejoperdomo68@gmail.com>; Andrew Linfoot <\r\nandrew.linfoot@gmail.com>; Andrew McPherson <andrewmcp@optusnet.com.au>;\r\nA&S <andsal@optusnet.com.au>; Andy Kuo <andy.kuo1@bayer.com>; Chi Ooi <\r\nchigooi@gmail.com>; Chris and Wendy Annable <cwannable@hotmail.com>;\r\nCourtenay Farquharson <courtenay.farquharson@gmail.com>; Daniel Lu <\r\nmail2danlu@gmail.com>; David Chen <dst.chen@gmail.com>; Dave Jackson <\r\nDAVID.A.JACKSON.1138@googlemail.com>; David Stanfield <\r\ndavid.stanfield@hotmail.com>; denis gradinscak <dgrad76@gmail.com>; Gavin\r\nGillett <gavin@gillettproperties.com.au>; Ian Partis <\r\nian_partis@optusnet.com.au>; James Ward <james.ward@nbrsap.com.au>; John\r\nCormican <johncormican@icloud.com>; Kerem Kaner <kaner.kerem@gmail.com>;\r\nMark Latham <mark.latham@bayer.com>; Maros Elsik <melsik@me.com>;\r\nmatt_bywaters@yahoo.co.uk; michael.aitkenhead@cantab.net; Michael Ee <\r\nDaddy_ee@hotmail.com>; Nick Benton <nzrheum@gmail.com>; Donnelley Paul\r\n(BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com>; Peter Janssen <\r\npj@janssen.net.au>; Richard Duggan <rdugganrduggan@yahoo.com.au>; robert\r\nhughes <robertthughes@yahoo.com>; Steve WIlson <wilson_steve@me.com>; Tung\r\nVu <tungvu@bigpond.com>; Tim Lu <tim.rheumatology@gmail.com>; Bennett,\r\nAndrew(Property) <Andrew.Bennett2@anz.com>; Victor Tiong <\r\nvictor.tiong@gmail.com>; YL Wang <wyl03@yahoo.com>; scott girard <\r\nscottrgirard@gmail.com>; Wilson Tehkh <wilson_tehkh@hotmail.com>\r\n*Subject:* Dangerous conditions for cycling Akuna (anticlockwise) this\r\nSunday 12th May\r\n\r\n\r\n\r\nDear Men,\r\n\r\n\r\n\r\nTreacherous conditions are predicted for cycling this Sunday 12th May\r\n(Mother\'s Day).  For those of you brave enough to use the Janssen v Janssen\r\n2014 defence "It\'s mother\'s day not wives\' day", please meet at 0700hrs at\r\nthe corner of Mawson St and Mona Vale Rd St Ives.  It\'s 35km to Akuna\r\n(anticlockwise) and back, with a cafe stop at JetBar at the end for those\r\nwith time.\r\n\r\n\r\n\r\nPlease RSVP if you are brave enough or not (I\'m out).\r\n\r\n\r\n\r\nCheers,\r\n\r\n\r\n\r\nGeorge\r\n\r\n\r\n-- \r\nRegards\r\n\r\nCourtenay Farquharson\r\n'

In [118]:
def find_nearby(email, email_body):
    start_pos = []
    number = []
    twitter = []
    facebook = []
    linkedin = []
    for match in re.finditer(email, email_body):
        start_pos.append(match.start())
    for j in range(len(start_pos)):
        end_pos = start_pos[j] + len(email)
        number.append(extract_phone_numbers(email_body[start_pos[j]-100:start_pos[j]]))
        number.append(extract_phone_numbers(email_body[end_pos:end_pos + 100]))
        twitter.append(extract_twitter(email_body[start_pos[j]-50:start_pos[j]]))
        twitter.append(extract_twitter(email_body[end_pos:end_pos + 100]))
        facebook.append(extract_facebook(email_body[start_pos[j]-50:start_pos[j]]))
        facebook.append(extract_facebook(email_body[end_pos:end_pos + 100]))
        linkedin.append(extract_linkedin(email_body[start_pos[j]-50:start_pos[j]]))
        linkedin.append(extract_linkedin(email_body[end_pos:end_pos + 100]))
        print(email, number,twitter,linkedin)

## Logic to get names and emails that belong to each other

In [48]:
data = '---------- Forwarded message ---------\r\nFrom: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com>\r\nDate: Fri, May 10, 2019 at 5:09 PM\r\nSubject: RE: Dangerous conditions for cycling Akuna (anticlockwise) this\r\nSunday 12th May\r\nTo: george lau <georgeandkim@gmail.com>, Adrian See <adriansee@hotmail.com>,\r\nAdrian Wong <adwong_au@yahoo.com>, Alex Perdomo <alejoperdomo68@gmail.com>,\r\nAndrew Linfoot <andrew.linfoot@gmail.com>, Andrew McPherson <\r\nandrewmcp@optusnet.com.au>, A&S <andsal@optusnet.com.au>, Andy Kuo <\r\nandy.kuo1@bayer.com>, Chi Ooi <chigooi@gmail.com>, Chris and Wendy Annable <\r\ncwannable@hotmail.com>, Courtenay Farquharson <\r\ncourtenay.farquharson@gmail.com>, Daniel Lu <mail2danlu@gmail.com>, David\r\nChen <dst.chen@gmail.com>, Dave Jackson <DAVID.A.JACKSON.1138@googlemail.com>,\r\nDavid Stanfield <david.stanfield@hotmail.com>, denis gradinscak <\r\ndgrad76@gmail.com>, Gavin Gillett <gavin@gillettproperties.com.au>, Ian\r\nPartis <ian_partis@optusnet.com.au>, James Ward <james.ward@nbrsap.com.au>,\r\nJohn Cormican <johncormican@icloud.com>, Kerem Kaner <kaner.kerem@gmail.com>,\r\nMark Latham <mark.latham@bayer.com>, Maros Elsik <melsik@me.com>,\r\nmatt_bywaters@yahoo.co.uk <matt_bywaters@yahoo.co.uk>,\r\nmichael.aitkenhead@cantab.net <michael.aitkenhead@cantab.net>, Michael Ee <\r\nDaddy_ee@hotmail.com>, Nick Benton <nzrheum@gmail.com>, Peter Janssen <\r\npj@janssen.net.au>, Richard Duggan <rdugganrduggan@yahoo.com.au>, robert\r\nhughes <robertthughes@yahoo.com>, Steve WIlson <wilson_steve@me.com>, Tung\r\nVu <tungvu@bigpond.com>, Tim Lu <tim.rheumatology@gmail.com>, Bennett,\r\nAndrew(Property) <Andrew.Bennett2@anz.com>, Victor Tiong <\r\nvictor.tiong@gmail.com>, YL Wang <wyl03@yahoo.com>, scott girard <\r\nscottrgirard@gmail.com>, Wilson Tehkh <wilson_tehkh@hotmail.com>\r\n\r\n\r\nPeter \xe2\x80\x93 I will see you there. Looks like a cheap coffee run afterwards ;-)\r\n\r\n\r\n\r\nBest regards,\r\n\r\n\r\n*Paul Donnelley *\r\nTel. +61298424752 | Mobile +61417696638\r\n\r\n*From:* george lau <georgeandkim@gmail.com>\r\n*Sent:* Friday, 10 May 2019 4:03 PM\r\n*To:* Adrian See <adriansee@hotmail.com>; Adrian Wong <adwong_au@yahoo.com>;\r\nAlex Perdomo <alejoperdomo68@gmail.com>; Andrew Linfoot <\r\nandrew.linfoot@gmail.com>; Andrew McPherson <andrewmcp@optusnet.com.au>;\r\nA&S <andsal@optusnet.com.au>; Andy Kuo <andy.kuo1@bayer.com>; Chi Ooi <\r\nchigooi@gmail.com>; Chris and Wendy Annable <cwannable@hotmail.com>;\r\nCourtenay Farquharson <courtenay.farquharson@gmail.com>; Daniel Lu <\r\nmail2danlu@gmail.com>; David Chen <dst.chen@gmail.com>; Dave Jackson <\r\nDAVID.A.JACKSON.1138@googlemail.com>; David Stanfield <\r\ndavid.stanfield@hotmail.com>; denis gradinscak <dgrad76@gmail.com>; Gavin\r\nGillett <gavin@gillettproperties.com.au>; Ian Partis <\r\nian_partis@optusnet.com.au>; James Ward <james.ward@nbrsap.com.au>; John\r\nCormican <johncormican@icloud.com>; Kerem Kaner <kaner.kerem@gmail.com>;\r\nMark Latham <mark.latham@bayer.com>; Maros Elsik <melsik@me.com>;\r\nmatt_bywaters@yahoo.co.uk; michael.aitkenhead@cantab.net; Michael Ee <\r\nDaddy_ee@hotmail.com>; Nick Benton <nzrheum@gmail.com>; Donnelley Paul\r\n(BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com>; Peter Janssen <\r\npj@janssen.net.au>; Richard Duggan <rdugganrduggan@yahoo.com.au>; robert\r\nhughes <robertthughes@yahoo.com>; Steve WIlson <wilson_steve@me.com>; Tung\r\nVu <tungvu@bigpond.com>; Tim Lu <tim.rheumatology@gmail.com>; Bennett,\r\nAndrew(Property) <Andrew.Bennett2@anz.com>; Victor Tiong <\r\nvictor.tiong@gmail.com>; YL Wang <wyl03@yahoo.com>; scott girard <\r\nscottrgirard@gmail.com>; Wilson Tehkh <wilson_tehkh@hotmail.com>\r\n*Subject:* Dangerous conditions for cycling Akuna (anticlockwise) this\r\nSunday 12th May\r\n\r\n\r\n\r\nDear Men,\r\n\r\n\r\n\r\nTreacherous conditions are predicted for cycling this Sunday 12th May\r\n(Mother\'s Day).  For those of you brave enough to use the Janssen v Janssen\r\n2014 defence "It\'s mother\'s day not wives\' day", please meet at 0700hrs at\r\nthe corner of Mawson St and Mona Vale Rd St Ives.  It\'s 35km to Akuna\r\n(anticlockwise) and back, with a cafe stop at JetBar at the end for those\r\nwith time.\r\n\r\n\r\n\r\nPlease RSVP if you are brave enough or not (I\'m out).\r\n\r\n\r\n\r\nCheers,\r\n\r\n\r\n\r\nGeorge\r\n\r\n\r\n-- \r\nRegards\r\n\r\nCourtenay Farquharson\r\n'
data = text_cleaning(data)
data

'---------- Forwarded message --------- From: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com> Date: Fri May 10 2019 at 5:09 PM Subject: RE: Dangerous conditions for cycling Akuna (anticlockwise) this Sunday 12th May To: george lau <georgeandkim@gmail.com> Adrian See <adriansee@hotmail.com> Adrian Wong <adwong_au@yahoo.com> Alex Perdomo <alejoperdomo68@gmail.com> Andrew Linfoot <andrew.linfoot@gmail.com> Andrew McPherson < andrewmcp@optusnet.com.au> A&S <andsal@optusnet.com.au> Andy Kuo < andy.kuo1@bayer.com> Chi Ooi <chigooi@gmail.com> Chris and Wendy Annable < cwannable@hotmail.com> Courtenay Farquharson < courtenay.farquharson@gmail.com> Daniel Lu <mail2danlu@gmail.com> David Chen <dst.chen@gmail.com> Dave Jackson <DAVID.A.JACKSON.1138@googlemail.com> David Stanfield <david.stanfield@hotmail.com> denis gradinscak < dgrad76@gmail.com> Gavin Gillett <gavin@gillettproperties.com.au> Ian Partis <ian_partis@optusnet.com.au> James Ward <james.ward@nbrsap.com.au> John C

In [ ]:
data.split('>')

In [14]:
import operator

In [44]:
def find_correct_name(names, email_addr):
    name_counter={name.title():0 for name in names}
    for name in name_counter:
        for item in name.split(' '):
            try:
                name_counter[name] += len(re.findall(item.lower(), email_addr.lower()))
            except Exception as e:
                print(e)
#     print(name_counter)
    name_counter = sorted(name_counter.items(), key=operator.itemgetter(1), reverse= True)
    if name_counter[0][1]>0:
        return name_counter[0][0]
#     return max(name_counter.items(), key=operator.itemgetter(1))[0]

In [151]:
def data_splitting(data):
    contacts=[]
    try:
        for line in data.split('>'):
            email_addrs = extract_email_addresses(line)
            name=None
            if len(email_addrs) == 1:
                names = extract_name(line)
                if len(names)>1:
                    name= find_correct_name(names, email_addrs[0])
                elif len(names) == 1:
                    name = names[0]
                else:
                    pass
            else:
                pass
            if name is not None:
#                 first_name = name.split(' ', 1)[0]
#                 last_name = name.split(' ', 1)[1]
                contacts.append([name.split(' ', 1), email_addrs[0]])
        return contacts
    except Exception as e:
        print(e)

In [146]:
name = 'Shoumik Goswami'
first_name = name.split(' ', 1)[0]
last_name = name.split(' ', 1)[1]

In [148]:
last_name

'Goswami'

In [107]:
data = 'Trail\xa0+1On Sun, May 19, 2019 at 11:27 AM Shoumik Goswami <shoumikgoswami@gmail.com> wrote:Shoumik Goswami---------- Forwarded message ---------From: Courtenay Farquharson <courtenay.farquharson@gmail.com>Date: Sat, May 18, 2019 at 4:17 PMSubject: Fwd: 119050425000150To:  <shoumikgoswami@gmail.com>---------- Forwarded message ---------From: Nasmin Kaisar (MINDTREE LIMITED) <v-naskai@microsoft.com>Date: Sat, May 18, 2019 at 9:08 AMSubject: RE: 119050425000150To: Courtenay Farquharson <courtenay.farquharson@gmail.com>Cc: Elyaas Mohammed (MINDTREE LIMITED) <v-elmoha@microsoft.com>\n\n\nHello Courtenay,\n\xa0\nI have processed the refund from my end .\r\n\nIt usually takes 7-10 business days for the refund to be credited back to your account.\r\n\n\xa0\nI will have to check with my internal team .\r\n\nBut since it is the weekend , there will be no update before next Monday.\r\n\nI have reopened the case.\r\n\n\xa0\nKind Regards,\n\nNasmin Kaisar,\xa0Support\r\n Engineer\n\nAzure Subscription Management Support\n\nEmail:\xa0v-naskai@microsoft.com\n\nManager:\xa0v-elmoha@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Tue - Sat) PT |\xa0Local\r\n Time\n\nTo reach Azure Support outside of my working hours, please email\xa0asmsmtbp@microsoft.com\n\nwith your support request number.\n\n\xa0\nService\r\n Dashboard\xa0|\xa0Create\r\n a Case\n\n\xa0\n\nAzure Support will never ask for your account password or full credit card number.\n\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\xa0\nFrom: Courtenay Farquharson [mailto:courtenay.farquharson@gmail.com]\r\n\nSent: Friday, May 17, 2019 3:56 PM\nTo: Elyaas Mohammed (MINDTREE LIMITED) <v-elmoha@microsoft.com>\nCc: Nasmin Kaisar (MINDTREE LIMITED) <v-naskai@microsoft.com>\nSubject: Re: 119050425000150\n\xa0\n\nHi Elyaas\n\n\xa0\n\n\nWe still havent seen any refund? It has been 7 business days since logged.\n\n\n\xa0\n\n\nCan you confirm it is still processing?\n\n\n\xa0\n\n\nOn Sat, 11 May 2019, 3:39 am Elyaas Mohammed (MINDTREE LIMITED), <v-elmoha@microsoft.com> wrote:\n\n\n\n\nHello Courtenay,\n\xa0\nMy name is Elyaas and I am one of Operations Manager here at Microsoft Support. You were working with one of my Engineer, Nasmin \xa0on the above mentioned case. I was reviewing your case and understand\r\n that we are ready to archive the case.\n\xa0\nBefore closing your case I wanted to take this opportunity to personally contact you to ensure that as a Microsoft customer you received the best possible support available and if there is any\r\n way we may have better assisted you during this support incident.\n\xa0\nYou may receive an invitation from Microsoft to complete a brief web survey. Your feedback is important to us. We use it to continually improve our service. We would appreciate it if you can\r\n take the time to complete this survey.\n\xa0\nI thank you for using Microsoft Products and Services. At Microsoft we strive to provide extraordinary customer service, so I look forward to hearing you.\n\xa0\nThank you once again for allowing us to assist you!\xa0 Have a great week!\n\xa0\n\xa0\nThanks!\nKind Regards,\nMohammed Elyaas,\r\nOperations Manager\nAzure Subscription Management Support\nEmail:\r\nv-elmoha@microsoft.com\nManager:\r\nv-vijaba@microsoft.com\nHours: 6:00 AM-3:00 PM (Mon - Fri) PT |\r\nLocal\r\n Time\nTo reach Azure Support outside of my working hours, please email to below alias\xa0with\r\n your support request number:\nAccounts & Portal issue :\r\nasmsmtap@microsoft.com\nBilling & Payment issue :\r\nasmsmtbp@microsoft.com\n\xa0\nService\r\n Dashboard |\r\nCreate\r\n a Case|\r\nAzure\r\n Announcements |\r\nHave\r\n an idea or a suggestion?\n\xa0\nAzure Support will never ask for your account password or full credit card number.\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\xa0\n\nKind Regards,\n\nMohammed Elyaas,\r\nOperations Manager\nAzure Subscription Management Support\nEmail:\r\nv-elmoha@microsoft.com\nManager:\r\nv-vijaba@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Mon - Fri) PT |\r\nLocal\r\n Time\n\nTo reach Azure Support outside of my working hours, please email to below alias\xa0with\r\n your support request number:\n\nAccounts & Portal issue :\r\nasmsmtap@microsoft.com\n\nBilling & Payment issue :\r\nasmsmtbp@microsoft.com\n\n\xa0\n\nService\r\n Dashboard |\r\nCreate\r\n a Case |\r\nAzure\r\n Announcements\n| Have\r\n an idea or a suggestion?\n\n\xa0\n\nAzure Support will never ask for your account password or full credit card number.\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\xa0\nHi\xa0Courtenay,\n\xa0\nSince there are no pending action from my end , I will now proceed to archive this case .\xa0\nPlease feel free to reopen the case in case if you do not receive the refund within 7-10 business days.\xa0\n\xa0\nTo reopen the case , please follow the below steps :\n\xa0\n1.Access the Azure portal (https://portal.azure.com/)\r\n\n2.Selecting "Manage support requests" in the top-right Help menu (?)\r\n\n3.Selecting your request\n4.Clicking “New Message” to re-open the request.\r\n\n\xa0\nIn the event that you find your request is not listed in the portal, please send an email to\r\nasmsmtbp@microsoft.com. We then will reopen the request and continue working on the issue.\xa0\nThanking you once again for all the support and understanding while we were working on the case.\n\xa0\nKind Regards,\n\nNasmin Kaisar,\xa0Support\r\n Engineer\n\nAzure Subscription Management Support\n\nEmail:\xa0v-naskai@microsoft.com\n\nManager:\xa0v-elmoha@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Tue - Sat) PT |\xa0Local\r\n Time\n\nTo reach Azure Support outside of my working hours, please email\xa0asmsmtbp@microsoft.com\n\nwith your support request number.\n\n\xa0\nService\r\n Dashboard\xa0|\xa0Create\r\n a Case\n\n\xa0\n\nAzure Support will never ask for your account password or full credit card number.\n\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\n\n\nFrom:\r\ncourtenay@parserr.com\nSent: :\r\nWed, May 8th 19, 12:58 pm\nTo:\r\nv-naskai@microsoft.com\nCc:\r\nsupport@mail.support.microsoft.com;courtenay.farquharson@gmail.com;v-moimt@microsoft.com;v-sajhin@microsoft.com;farqy1@hotmail.com;v-elmoha@microsoft.com\nSubject\r\nRe: Re: 119050425000150\n\xa0\nNo problems.\r\n\n\xa0\nIll keep an eye out for it.\xa0\n\xa0\nOn Thu, 9 May 2019, 5:54 am Nasmin Kaisar (MINDTREE LIMITED), <v-naskai@microsoft.com>\r\n wrote:\nHi\xa0Courtenay,\n\xa0\nIt will go back to the credit card and it should reflect on the credit card statement .\r\n\nDo you have any further concerns regarding this issue?\n\xa0\nKind Regards,\nNasmin Kaisar,\xa0Support\r\n Engineer\nAzure Subscription Management Support\nEmail:\xa0v-naskai@microsoft.com\nManager:\xa0v-elmoha@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Tue - Sat) PT |\xa0Local\r\n Time\nTo reach Azure Support outside of my working hours, please email\xa0asmsmtbp@microsoft.com\nwith your support request number.\n\n\xa0\nService\r\n Dashboard\xa0|\xa0Create\r\n a Case\n\n\xa0\nAzure Support will never ask for your account password or full credit card number.\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\xa0\n\xa0\n\xa0\nFrom: Courtenay Farquharson [mailto:courtenay@parserr.com]\r\n\nSent: Wednesday, May 8, 2019 12:48 PM\nTo: support <support@mail.support.microsoft.com>; Nasmin Kaisar (MINDTREE LIMITED) <v-naskai@microsoft.com>\nCc: Courtenay Farquharson <courtenay.farquharson@gmail.com>; Mohammed Imtiyaz Ali (MINDTREE LIMITED) <v-moimt@microsoft.com>;\r\n Saikumar Jhingade (MINDTREE LIMITED) <v-sajhin@microsoft.com>;\r\n\r\nfarqy1@hotmail.com; Elyaas Mohammed (MINDTREE LIMITED) <v-elmoha@microsoft.com>\nSubject: Re: Re: 119050425000150\n\xa0\nWill the refund be reflected in my Azure account?\n\xa0\nOn Thu, 9 May 2019, 5:43 am , <v-naskai@microsoft.com>\r\n wrote:\nHi\xa0Courtenay,\n\xa0\nYou will be receiving the amount within 7-10 business days.\xa0\nThat is our standard time frame for the refund to process.\xa0\xa0\n\xa0\nIs there anything else I can help you\xa0with? If not, then can I archive this case?\n\xa0\nKind Regards,\nNasmin Kaisar,\xa0Support\r\n Engineer\nAzure Subscription Management Support\nEmail:\xa0v-naskai@microsoft.com\nManager:\xa0v-elmoha@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Tue - Sat) PT |\xa0Local\r\n Time\nTo reach Azure Support outside of my working hours, please email\xa0asmsmtbp@microsoft.com\nwith your support request number.\n\n\xa0\nService\r\n Dashboard\xa0|\xa0Create\r\n a Case\n\n\xa0\nAzure Support will never ask for your account password or full credit card number.\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\xa0\n\n\n\nFrom:\r\ncourtenay.farquharson@gmail.com\nSent: :\r\nWed, May 8th 19, 12:13 pm\nTo:\r\nsupport@mail.support.microsoft.com;v-naskai@microsoft.com\nCc:\r\nv-moimt@microsoft.com;v-sajhin@microsoft.com;courtenay@parserr.com;farqy1@hotmail.com;v-elmoha@microsoft.com\nSubject\r\nRe: 119050425000150\n\xa0\nThank you.\r\n\n\xa0\nDo we know when the refund qill be processed?\n\xa0\nOn Thu, 9 May 2019, 1:59 am , <v-naskai@microsoft.com>\r\n wrote:\nHi\xa0Courtenay,\n\xa0\nI hope this email finds you well. I \xa0haven’t heard back from you, so just wanted to check if you needed any further assistance.\xa0 Please do let me know.\nKind Regards,\nNasmin Kaisar,\xa0Support\r\n Engineer\nAzure Subscription Management Support\nEmail:\xa0v-naskai@microsoft.com\nManager:\xa0v-elmoha@microsoft.com\n\nHours: 6:00 AM-3:00 PM (Tue - Sat) PT |\xa0Local\r\n Time\nTo reach Azure Support outside of my working hours, please email\xa0asmsmtbp@microsoft.com\nwith your support request number.\n\n\xa0\nService\r\n Dashboard\xa0|\xa0Create\r\n a Case\n\n\xa0\nAzure Support will never ask for your account password or full credit card number.\nFor your privacy and security, NEVER send your login credentials or full credit card number in email.\n\xa0\n\n\n\n\n\n\n-- RegardsCourtenay Farquharson\n\n\n'

In [51]:
data.split('>')

['---------- Forwarded message --------- From: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com',
 ' Date: Fri May 10 2019 at 5:09 PM Subject: RE: Dangerous conditions for cycling Akuna (anticlockwise) this Sunday 12th May To: george lau <georgeandkim@gmail.com',
 ' Adrian See <adriansee@hotmail.com',
 ' Adrian Wong <adwong_au@yahoo.com',
 ' Alex Perdomo <alejoperdomo68@gmail.com',
 ' Andrew Linfoot <andrew.linfoot@gmail.com',
 ' Andrew McPherson < andrewmcp@optusnet.com.au',
 ' A&S <andsal@optusnet.com.au',
 ' Andy Kuo < andy.kuo1@bayer.com',
 ' Chi Ooi <chigooi@gmail.com',
 ' Chris and Wendy Annable < cwannable@hotmail.com',
 ' Courtenay Farquharson < courtenay.farquharson@gmail.com',
 ' Daniel Lu <mail2danlu@gmail.com',
 ' David Chen <dst.chen@gmail.com',
 ' Dave Jackson <DAVID.A.JACKSON.1138@googlemail.com',
 ' David Stanfield <david.stanfield@hotmail.com',
 ' denis gradinscak < dgrad76@gmail.com',
 ' Gavin Gillett <gavin@gillettproperties.com.au',
 ' Ian Partis 

In [56]:
for line in data.split('>'):
    email_addrs = extract_email_addresses(line)
    name=None
    if len(email_addrs) == 1:
        names = extract_name(line)
        if len(names)>1:
            name= find_correct_name(names, email_addrs[0])
        elif len(names) == 1:
            name = names[0]
        else:
            pass
    else:
        pass
    if name is not None:
        print([name, email_addrs[0]])
        print()

['Donnelley Paul', 'Paul.Donnelley@au.bosch.com']

['George Lau', 'georgeandkim@gmail.com']

['Adrian Wong', 'adwong_au@yahoo.com']

['Alex Perdomo', 'alejoperdomo68@gmail.com']

['Andrew Linfoot <', 'andrew.linfoot@gmail.com']

['Andrew McPherson', 'andrewmcp@optusnet.com.au']



KeyboardInterrupt: 

Testing 

In [37]:
data1 = '---------- Forwarded message ---------From: Nasmin Kaisar (MINDTREE LIMITED) <v-naskai@microsoft.com>'
email_addrs = extract_email_addresses(data1)
email_addrs
extract_name(data1)

['Nasmin Kaisar']

In [54]:
names = ['Bennett Andrew(Property)', 'A&S']
email_addr='andsal@optusnet.com.au'
name_counter={name.title():0 for name in names}
print(name_counter)
for name in name_counter:
    for item in name.split(' '):
        name_counter[name] += len(re.findall(item.lower(), email_addr))

name_counter = sorted(name_counter.items(), key=operator.itemgetter(1), reverse= True)
print(name_counter)
if name_counter[0][1]>0:
        print(name_counter[0][0])
#     return max(name_counter.items(), key=operator.itemgetter(1))[0]

{'Bennett Andrew(Property)': 0, 'A&S': 0}
[('Bennett Andrew(Property)', 0), ('A&S', 0)]


In [ ]:
re.findall('Bennett'.lower(), 'Andrew.Bennett2@anz.com'.lower())